In [72]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle
from google.auth.exceptions import RefreshError
import logging
import pandas as pd 

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [73]:
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = r'D:\2A\2A\mods207\code_secret_client.json'
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if creds and creds.refresh_token:
        try:
            creds.refresh(Request())
        except RefreshError:
            logger.error("Credentials could not be refreshed, possibly the authorization was revoked by the user.")
            os.unlink('token.pickle')
            return
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            client_secrets_file, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
            
            


    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

In [74]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [75]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [81]:
def save_video_infos(video_response, df):
    items = video_response.get("items")[0]
    # get the snippet, statistics & content details from the video response
    snippet         = items["snippet"]
    statistics      = items["statistics"]
    content_details = items["contentDetails"]
    # get infos from the snippet
    channel_title = snippet["channelTitle"]
    title         = snippet["title"]
    description   = snippet["description"]
    publish_time  = snippet["publishedAt"]
    if "tags" in snippet : 
        key_words = snippet["tags"]
    else :
        key_words = []
        
    category      = snippet["categoryId"]
    # get stats infos
    if "commentCount" in statistics : 
        comment_count = statistics["commentCount"]
    else : 
        comment_count = -1

    if "likeCount" in statistics :
        like_count = statistics["likeCount"]
    else :
        like_count = -1
        
    favorite_count = statistics["favoriteCount"]
    view_count     = statistics["viewCount"]    
    # get duration from content detail  
    duration   = content_details["duration"]
    definition = content_details["definition"]
    caption    = content_details["caption"]
    licensed   = content_details["licensedContent"]
    # duration in the form of something like 'PT5H50M15S'
    # parsing it to be something like '5:50:15'
    parsed_duration = re.search(f"PT(\d+H)?(\d+M)?(\d+S)", duration)
    
    duration_str = ""
    if parsed_duration != None :
        for d in parsed_duration.groups():
            if d:
                duration_str += f"{d[:-1]}:"
        duration_str = duration_str.strip(":")
    
    df = df.append({"Title": title, "Description" : description, "Channel Title" : channel_title,
                    "Publish time" : publish_time, "Tags": key_words, "Category" : category, 
                    "Duration" : duration_str, "Number of comments" : comment_count, 
                    "Number of likes" : like_count, "Number of views" : view_count, 
                    "Number of favorites" : favorite_count, "Video quality" : definition,
                    "Licensed" : licensed} ,   ignore_index=True)
    return df

# Comments

In [56]:
def get_comments(youtube, **kwargs):
    return youtube.commentThreads().list(
        part="snippet",
        **kwargs
    ).execute()

In [7]:
# URL can be a channel or a video, to extract comments
url = "https://www.youtube.com/watch?v=CpZBJZYJKFE&ab_channel=Cyprien"
if "watch" in url:
    # that's a video
    video_id = get_video_id_by_url(url)
    params = {
        'videoId': video_id, 
        'maxResults': 2,
        'order': 'relevance', # default is 'time' (newest)
    }
else:
    # should be a channel
    channel_id = get_channel_id_by_url(url)
    params = {
        'allThreadsRelatedToChannelId': channel_id, 
        'maxResults': 2,
        'order': 'relevance', # default is 'time' (newest)
    }
# get the first 2 pages (2 API requests)
n_pages = 2
for i in range(n_pages):
    # make API call to get all comments from the channel (including posts & videos)
    response = get_comments(youtube, **params)
    items = response.get("items")
    # if items is empty, breakout of the loop
    if not items:
        break
    for item in items:
        comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        updated_at = item["snippet"]["topLevelComment"]["snippet"]["updatedAt"]
        like_count = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
        comment_id = item["snippet"]["topLevelComment"]["id"]
        print(f"""\
        Comment: {comment}
        Likes: {like_count}
        Updated At: {updated_at}
        ==================================\
        """)
    if "nextPageToken" in response:
        # if there is a next page
        # add next page token to the params we pass to the function
        params["pageToken"] =  response["nextPageToken"]
    else:
        # must be end of comments!!!!
        break
    print("*"*70)

        Comment: N&#39;oubliez pas d&#39;utiliser mon lien pour la réduction sur l&#39;abonnement de 2 ans à NordVPN <a href="http://www.nordvpn.com/cyprien">http://www.nordvpn.com/cyprien</a> et merci pour vos messages ✌
        Likes: 3142
        Updated At: 2022-04-29T15:10:23Z
        Comment: Comme toujours : Masterclass. Quelle créativité folle, ça fait du bien de voir ça sur Youtube !! :)
        Likes: 9310
        Updated At: 2022-04-29T15:15:24Z
**********************************************************************
        Comment: Toujours un plaisir de voir à quel point tu arrive à crée des personnages attachant sur un format si court, toujours plein de créativité, un vrai plaisir d’avoir ce genre de contenus sur YouTube.
        Likes: 259
        Updated At: 2022-05-04T04:05:34Z
        Comment: Masterclass de Cyprien comme d’habitude c’est créatif et innovant, un petit scénario derrière c’est super on passe toujours un bon moment.
        Likes: 442
        Updated At: 

In [93]:
def get_channel_videos(path, channel):
    
    channel_csv = pd.read_csv(path + channel)
    channel_csv.drop_duplicates(inplace=True)  # drop ducplicates 
    channel_csv.dropna(inplace=True)  # drop nan
    videos = channel_csv['videos'].apply(lambda x : str(x)) # all video urls are an str type
    videos = videos[videos.apply(lambda x : "https://www.youtube.com/watch?v" in x)] # keep only videos
    video_ids = videos.apply(lambda x : get_video_id_by_url(x)) # extract video id
    
    df = pd.DataFrame() 
    for video_id in video_ids :
        try :
            response = get_video_details(youtube, id=video_id)
            df = save_video_infos(response, df)
            #print("got details for", video_id)
            
        except Exception as inst:
            print(type(inst))    # the exception instance
            print(inst) 
            print("Error in video id ", video_id)
    
    return df 

In [94]:
path = 'channelVideos/'
all_videos = os.listdir(path)

for channel in all_videos :
    df = get_channel_videos(path, channel)
    channel_videos = 'channel' + channel.split("_")[1] + 'videos.csv'
    df.to_csv(channel_videos, index=False)
    print('scrapped ', channel)
    


scrapped  channel_0_UCWeg2Pkate69NFdBeuRFTAw.csv
scrapped  channel_10_UCp8Lmjx1i93YVokA4fNAs1g.csv
scrapped  channel_11_UCgvqvBoSHB1ctlyyhoHrGwQ.csv
scrapped  channel_12_UCo3i0nUzZjjLuM7VjAVz4zA.csv
scrapped  channel_13_UCmPSwsooZq8an7xOLQQhAdw.csv
scrapped  channel_14_UCDPK_MTu3uTUFJXRVcTJcEw.csv
scrapped  channel_15_UCt6IQpsggvn6zmalhPglSEA.csv
scrapped  channel_16_UC-69vhXlCa3XHbF8JHCQHfg.csv


<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=lNyeQ60lPG4&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  lNyeQ60lPG4
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=gKD1ueEoTYk&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=IMfo6xQPIeA&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  IMfo6xQPIeA
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=D87MrO3E5M4&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=tJWh0hzqsMo&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  tJWh0hzqsMo
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=yTsN6Dll7oA&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=m0g2c63Kayo&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  m0g2c63Kayo
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=CJYFX1zLrYQ&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

...

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=W1zIcS8O2vc&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  W1zIcS8O2vc
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=3tFAHeIrGro&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=QH2qRwizUcw&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  QH2qRwizUcw
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=flgzxuZeFQI&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=tsuh9ZVVjIE&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  tsuh9ZVVjIE
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=3ND_joCGrY4&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=Q_lppH1-DuE&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">
Error in video id  Q_lppH1-DuE
<class 'googleapiclient.errors.HttpError'>
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=RRN5keKCqzw&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quot

KeyboardInterrupt: 